<a href="https://colab.research.google.com/github/seftiwulandari/sentiment-analysis/blob/main/SVM_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Support Vector Machine
oleh:
1. Istiqamatul Badriah (G1A017013)
2. Sefti Wulandari (G1A017016)

In [ ]:
pip install sastrawi

     |████████████████████████████████| 215kB 5.8MB/s 


In [ ]:
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('average_perceptron_tagger')
nltk.download('punkt')

import io
import unicodedata
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import  LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import wordcloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import  BeautifulSoup
import spacy
import pickle
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import wordnet
from sklearn import svm 
from sklearn.metrics import  classification_report,confusion_matrix,accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seftiwulandari","key":"0f096ee705baeea596de6d47c05d2671"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d jessemostipak/animal-crossing

  0% 0.00/754k [00:00<?, ?B/s]
100% 754k/754k [00:00<00:00, 50.0MB/s]


In [10]:
import zipfile
zip_ref = zipfile.ZipFile('animal-crossing.zip','r')
zip_ref.extractall('files')
zip_ref.close()

In [11]:
file_path ='/content/files/user_reviews.csv'
data = pd.read_csv(file_path)
data.head()

,grade,user_name,text,date
0,4,mds27272,My gf started playing before me. No option to ...,2020-03-20
1,5,lolo2178,"While the game itself is great, really relaxin...",2020-03-20
2,0,Roachant,My wife and I were looking forward to playing ...,2020-03-20
3,0,Houndf,We need equal values and opportunities for all...,2020-03-20
4,0,ProfessorFox,BEWARE! If you have multiple people in your h...,2020-03-20


In [12]:
# Merubah keseluruhan kalimat di kolom yang dipilih menjadi huruf kecil
data['text'] = data['text'].str.lower()
data.head()

,grade,user_name,text,date
0,4,mds27272,my gf started playing before me. no option to ...,2020-03-20
1,5,lolo2178,"while the game itself is great, really relaxin...",2020-03-20
2,0,Roachant,my wife and i were looking forward to playing ...,2020-03-20
3,0,Houndf,we need equal values and opportunities for all...,2020-03-20
4,0,ProfessorFox,beware! if you have multiple people in your h...,2020-03-20


In [13]:
y=data.grade
y.head()

0    4
1    5
2    0
3    0
4    0
Name: grade, dtype: int64

In [14]:
X_test = data.text.astype(str)
X_test.head()

0    my gf started playing before me. no option to ...
1    while the game itself is great, really relaxin...
2    my wife and i were looking forward to playing ...
3    we need equal values and opportunities for all...
4    beware!  if you have multiple people in your h...
Name: text, dtype: object

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("indonesian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import re
def preprocess(text):
  text.lower()
  text  = re.sub('((wwww\.[^\s]+)|(https?:[^s]+))','URL',text)
  text = re.sub('@[^\s]+','AT_USER',text)
  text = re.sub('[\s]+',' ',text)
  text = re.sub(r'#([\s]+)',r'\1',text)
  tokens = word_tokenize(text)
  tokens = [w for w in tokens if not w in stop_words]
  return " ".join(tokens)

X = X_test.apply(preprocess)
X.head()

0    my gf started playing before me . no option to...
1    while the game itself is great , really relaxi...
2    my wife and i were looking forward to playing ...
3    we need equal values and opportunities for all...
4    beware ! if you have multiple people in your h...
Name: text, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
def feature_extraction(data):
    tfv=TfidfVectorizer(sublinear_tf=True, stop_words = "english")
    features=tfv.fit_transform(data)
    pickle.dump(tfv.vocabulary_, open("svm_feature.pkl", "wb"))
    return features

data = np.array(X)
label = np.array(y)
features = feature_extraction(data)

print(features)

  (0, 10467)	0.21474149461442496
  (0, 3660)	0.12035699843043875
  (0, 4233)	0.05355580109435677
  (0, 10921)	0.1862973458698189
  (0, 4906)	0.1764858330430266
  (0, 7338)	0.13548389544500442
  (0, 1444)	0.11542316302466461
  (0, 9949)	0.183259907486867
  (0, 230)	0.1610853951605073
  (0, 8438)	0.1725907860233391
  (0, 317)	0.24109309491127537
  (0, 9939)	0.26227375542946196
  (0, 4335)	0.15936314874508148
  (0, 56)	0.23116038948892648
  (0, 6377)	0.23116038948892648
  (0, 9669)	0.19745251006692938
  (0, 2096)	0.169897885466961
  (0, 9462)	0.1621976859375903
  (0, 7461)	0.17451614377779995
  (0, 92)	0.3635889561556423
  (0, 4573)	0.24707026992386946
  (0, 5400)	0.06835199781700918
  (0, 2289)	0.17434504232800851
  (0, 6985)	0.1862973458698189
  (0, 7472)	0.11892450656909653
  :	:
  (2997, 5794)	0.36917591409025047
  (2997, 5448)	0.26779894310342944
  (2997, 3738)	0.2318799147909403
  (2997, 1667)	0.2946706062742537
  (2997, 4181)	0.18375522917443024
  (2997, 5822)	0.15965453961480197
 

In [18]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size = 0.20)

In [21]:
from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  

svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
from sklearn.metrics import accuracy_score
val_pred = svclassifier.predict(X_test)
#print(val_pred)
print(accuracy_score(y_test, val_pred))

0.595


In [23]:
filename = 'svm_model.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [24]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.595


In [25]:
text = 'while the game itself is great'
text = preprocess(text)
print(text)
text = np.array([text])
print(text)

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfv_loaded = TfidfVectorizer(sublinear_tf=True, stop_words = "english", vocabulary=pickle.load(open("svm_feature.pkl", "rb")))
text = transformer.fit_transform(tfv_loaded.fit_transform(text))
print(text)
polarity = loaded_model.predict(text)
print(polarity)

while the game itself is great
['while the game itself is great']
  (0, 4233)	0.7071067811865476
  (0, 4488)	0.7071067811865476
[10]
